This is combined code of both the members, as we tried different descripter to explore different techniques and analyze the accuracy.From here on, Canny edge detector with flattening is used to extract the feature of the images. Later we will see the results for HOG.

In [45]:
# Importing libraries

import numpy as np
from keras.preprocessing import image
import cv2 as cv
from pathlib import Path
from sklearn.model_selection import train_test_split
from skimage.io import imread
import matplotlib.pyplot as plt
import os
from skimage import feature
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [32]:
def process(container_path): # to load images files
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]
    train_img = []
    counts = []
    for i, direc in enumerate(folders):
        s = 0
        for file in direc.iterdir():
            s += 1
            img = imread(file)
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            img_pred = cv.resize(img, (50, 50), interpolation=cv.INTER_AREA)
            img_pred = image.img_to_array(img_pred)
            img_pred = img_pred / 255
            train_img.append(img_pred)
        counts.append(s)
    
    X = np.array(train_img)
    return X,counts,categories

In [36]:
# Path to the dataset
paths = r"D:\DOC\COLLEGE DOC\NOTES\SEMESTER 4\Data analysis\fr\fruits-360"

In [39]:
# Training dataset
X_train,counts,categories = process(paths+"\Training")
y_train = []
for i,a in enumerate(counts):
    b = np.zeros(a) + i
    y_train = np.concatenate((y_train,b), axis=0)

print("Trainig Files imported successfully")
print("X train of lenght",len(X_train))
print("The number of images in each folders are", counts)

# Testing dataset
X_test,counts,categories = process(paths+"\Test")
y_test = []
for i,a in enumerate(counts):
    b = np.zeros(a) + i
    y_test = np.concatenate((y_test,b), axis=0)
    
print("Test Files imported successfully")
print("X test of lenght",len(X_test))
print("The number of images in each folders are", counts)

Trainig Files imported successfully
X train of lenght 6163
The number of images in each folders are [492, 444, 427, 490, 450, 490, 492, 490, 468, 490, 490, 490, 450]
Test Files imported successfully
X test of lenght 1921
The number of images in each folders are [164, 148, 143, 166, 166, 164, 166, 156, 166, 166, 166, 150]


In [40]:
print("The number of fruits categories are", len(categories))
print("Total number of images are", len(X_train)+len(y_train))

The number of fruits categories are 12
Total number of images are 12326


In [41]:
print("X_train has a shape of", X_train.shape)
print("X_test has a shape of", X_test.shape)

print("y_train has a shape of", y_train.shape)
print("y_test has a shape of", y_test.shape)

num_training = X_train.shape[0]
num_test = X_test.shape[0]

# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

print("\nAfter proper flatenning of the image array in a single dimension of length",len(X_train[0]))
print("X_train has a shape of", X_train.shape)
print("X_test has a shape of", X_test.shape)

print("y_train has a shape of", y_train.shape)
print("y_test has a shape of", y_test.shape)

X_train has a shape of (6163, 50, 50, 1)
X_test has a shape of (1921, 50, 50, 1)
y_train has a shape of (6163,)
y_test has a shape of (1921,)

After proper flatenning of the image array in a single dimension of length 2500
X_train has a shape of (6163, 2500)
X_test has a shape of (1921, 2500)
y_train has a shape of (6163,)
y_test has a shape of (1921,)


In [42]:
print("The fruits in the training set are\n")
categories

The fruits in the training set are



['Apple Braeburn',
 'Apple Crimson Snow',
 'Avocado',
 'Banana',
 'Cactus fruit',
 'Cantaloupe 1',
 'Dates',
 'Eggplant',
 'Grape White',
 'Guava',
 'Passion Fruit',
 'Potato Red']

In [43]:
class KNN(object):
    
    def __init__(self,fr,label):
        self.X_train = fr
        self.y_train = label

    def L2_distances(self, Xtest):
        num_test = Xtest.shape[0]
        num_train = self.X_train.shape[0]
        distances = np.zeros((num_test, num_train))
        dot = Xtest.dot(self.X_train.T)
        distances = np.sqrt((Xtest ** 2).sum(axis=1, keepdims=1) + (self.X_train ** 2).sum(axis=1) - 2 * dot)
        return distances

    def predict(self, dists, k=1):
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            closest_y = self.y_train[np.argsort(dists[i])][0:k]
            #print(closest_y)
            closest_y = closest_y.astype(int)
            y_pred[i] = np.bincount(closest_y).argmax()
        return y_pred

In [44]:
classifier = KNN(X_train, y_train)

distances = classifier.L2_distances(X_test)
y_val_pred = classifier.predict(distances, k=2)
num_correct = np.sum(y_val_pred == y_test)
accuracy = num_correct / num_test

print("Accuracy is",accuracy)

Accuracy is 0.30400832899531494


In [55]:
acc = num_correct / num_test
prec = precision_score(y_test,y_val_pred,average='weighted')
recall = recall_score(y_test,y_val_pred,average='weighted',zero_division=1)
f1score = f1_score(y_test,y_val_pred,average='weighted')
print("Accuracy:",acc)
print("Precision:",prec)
print("Recall:",recall)
print("F1 Score:",f1score)

Accuracy: 0.30400832899531494
Precision: 0.32129175240331737
Recall: 0.30400832899531494
F1 Score: 0.30971196136259677


The expected average accuracy is very low, as consedering only edge detection; many fruits will resembel in outer shape and thus the accuracy score drops.

To improve the results we will use HOG descriptor.

In [27]:
# Path to the dataset
r = "D:/DOC/COLLEGE DOC/NOTES/SEMESTER 4/Data analysis/fr/fruits-360"

In [28]:
image_paths = os.listdir(f"{r}/Training")
images = []
for path in image_paths:
    all_images = os.listdir(f"{r}/Training/{path}")
    for image in all_images:
        image_path = f"{r}/Training/{path}/{image}"
        image = cv.imread(image_path)
        image = cv.resize(image, (64,64))
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        # HOG descriptor for the image
        hog_desc = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
        images.append([hog_desc,path])
    print(path)

Apple Braeburn
Apple Crimson Snow
Avocado
Banana
Beetroot
Cactus fruit
Cantaloupe 1
Dates
Eggplant
Grape White
Guava
Passion Fruit
Potato Red


In [62]:
test_image_paths = os.listdir(f"{r}/Test")
prediction=[]
test_label= []
y1_test=[]
y1_pred=[]
for path in test_image_paths:

    all_test_images = os.listdir(f"{r}/Test/{path}")

    image_pred=[]
    for image in all_test_images:
        image_path = f"{r}/Test/{path}/{image}"
        image = cv.imread(image_path)
        image = cv.resize(image, (64, 64))
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        # HOG descriptor for the image
        hog_desc = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')

        dist=[]
        for i in images:
            dist.append((np.linalg.norm(i[0]-hog_desc),i[1]))
        dist=sorted(dist)
        dist=dist[:5]
        count=dict()
        for i in range(5):
            count[dist[i][1]]=count.get(dist[i][1],0)+1
        ans=[0,0]
        for k in count:
            if ans[1]<count[k]:
                ans[0]=k
                ans[1]=count[k]
        image_pred.append(ans[0])
        y1_pred.append(ans[0])
        y1_test.append(path)
    prediction.append(image_pred)
    print(path)
    test_label.append(path)

Apple Braeburn
Apple Crimson Snow
Avocado
Banana
Cactus fruit
Cantaloupe 1
Dates
Eggplant
Grape White
Guava
Passion Fruit
Potato Red


The training time increase very much as compared to prototype based learning, as we are taking distance from each training image.

In [63]:
#class wise accuracy
def accuracy(pred,real):
    if len(real)==1:
        real=real*len(pred)
    return accuracy_score(real,pred)
s=0
for i in range(len(test_label)):
    acc = accuracy(prediction[i],[test_label[i]])
    s = s + acc
    print(f"Accuracy for the prediction of class {test_label[i]} is ",acc)
print("\nAverage is ",s/len(test_label))

Accuracy for the prediction of class Apple Braeburn is  0.8841463414634146
Accuracy for the prediction of class Apple Crimson Snow is  1.0
Accuracy for the prediction of class Avocado is  0.8461538461538461
Accuracy for the prediction of class Banana is  0.7650602409638554
Accuracy for the prediction of class Cactus fruit is  0.8975903614457831
Accuracy for the prediction of class Cantaloupe 1 is  1.0
Accuracy for the prediction of class Dates is  1.0
Accuracy for the prediction of class Eggplant is  0.7692307692307693
Accuracy for the prediction of class Grape White is  0.9337349397590361
Accuracy for the prediction of class Guava is  1.0
Accuracy for the prediction of class Passion Fruit is  0.927710843373494
Accuracy for the prediction of class Potato Red is  0.6733333333333333

Average is  0.8914133896436277


In [67]:
acc = accuracy_score(y1_test,y1_pred)
prec = precision_score(y1_test,y1_pred,average='weighted')
recall = recall_score(y1_test,y1_pred,average='weighted',zero_division=1)
f1score = f1_score(y1_test,y1_pred,average='weighted')
print("Accuracy:",acc)
print("Precision:",prec)
print("Recall:",recall)
print("F1 Score:",f1score)

Accuracy: 0.8932847475273296
Precision: 0.923782873308579
Recall: 0.8932847475273296
F1 Score: 0.9024271190347646


Moreover the relative accuracy increases in case of KNN as compared to prototype learning.
As from the first part we acheived the accuracy of 0.49 in case of prototype learning, but in this case KNN give better results. This is compoaritive results based on classification technique.

CONCLUSION: KNN is better approach to achieve higher accuracy but is not very time efficient as compared to prototype learning.